# Data Cleaning, Harmonization, and Exploratory Data Analysis

## Setup

### Import libraries

In [1]:
# standard library
from concurrent import futures
import logging
import multiprocessing as mp
from pathlib import Path
from typing import Callable, Optional, Sequence, Tuple, Iterable, List

# third-party libraries
import colorcet as cc
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import SaltRemover, Descriptors, rdMolDescriptors
from scipy import stats
from tqdm.auto import tqdm

# local libraries
from admet.data.chem import (
    parallel_canonicalize_smiles,
    compute_molecular_properties,
)
from admet.data.constants import (
    TRANSFORMATIONS,
    COLS_WITH_UNITS,
)
from admet.visualize.plots import (
    plot_numeric_distributions,
    plot_correlation_matrix,
    plot_property_distributions,
)

ModuleNotFoundError: No module named 'colorcet'

In [ ]:
# setup tqdm
tqdm.pandas()

In [ ]:
# setup logging
level = logging.DEBUG
logger = logging.getLogger(__name__)
if logger.hasHandlers():
    logger.handlers.clear()

formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel(level)

logger.info("Imports successful.")


## Load Data

In [ ]:
cleaned_data = {}

In [ ]:
# project data root directory
base_data_dir = Path().cwd().parents[0] / "assets/dataset/raw"

if not base_data_dir.exists():
    raise FileNotFoundError(f"Data directory not found at {base_data_dir}")

logger.info(f"Data directory found at {base_data_dir}")
for dataset_dir in base_data_dir.iterdir():
    logger.info(f"Dataset name: {dataset_dir.name}")

### Challenge Data: ExpansionRX

In [ ]:
# load data
data_path = base_data_dir / "ExpansionRX/full/expansion_data_train.csv"
df_expansionrx = pd.read_csv(data_path)

# rename cols to have units
df_expansionrx.rename(
    columns={k: f"{k} {v}" for k, v in COLS_WITH_UNITS.items() if k in df_expansionrx.columns},
    inplace=True,
)

# drop rows with invalid SMILES
df_expansionrx["SMILES"] = parallel_canonicalize_smiles(
    df_expansionrx["SMILES"].tolist(),
    isomeric=True,
)
n_none_smiles = df_expansionrx["SMILES"].isnull().sum()
if n_none_smiles > 0:
    logger.warning(f"Dropping {n_none_smiles} rows with invalid SMILES")
    df_expansionrx.dropna(subset=["SMILES"], inplace=True)
    df_expansionrx.reset_index(drop=True, inplace=True)

logger.info(f"Dataframe shape: {df_expansionrx.shape}")
logger.info(f"Dataframe columns: {df_expansionrx.columns.tolist()}")

# is the teaser data a subset of the full data?
merged_df = df_expansionrx.merge(
    df_expansionrx,
    on=["SMILES"] + [col for col in df_expansionrx.columns if col != "SMILES"],
    how="inner",
)
if merged_df.shape[0] == df_expansionrx.shape[0]:
    logger.info("The teaser data is a subset of the full data.")

In [ ]:
df_expansionrx.sort_values("Molecule Name (None)", inplace=True)
df_expansionrx.reset_index(drop=True, inplace=True)

df_expansionrx

In [ ]:
cleaned_data["expansionrx"] = df_expansionrx.copy()

In [ ]:
df = df_expansionrx.copy()
output_dir = data_path.parents[1] / "figures"
fname = "expansionrx"

df_props = compute_molecular_properties(df["SMILES"])
output_dir.mkdir(parents=True, exist_ok=True)
fig, ax = plot_numeric_distributions(
    df,
    n_cols=3,
    title="ExpansionRx Teaser Property Distributions",
    save_path=output_dir / f"{fname}_numeric_distributions.png",
)
fig, ax = plot_property_distributions(
    df_props,
    save_path=output_dir / f"{fname}_molecular_property_distributions.png",
)
fig, ax = plot_correlation_matrix(
    df,
    save_path=output_dir / f"{fname}_numeric_correlation_matrix.png",
)

### KERMT Dataset

#### Public

Key dataset information

In [ ]:
files_biogen = [
    "KERMT/export_public_cluster_split/all_test_cluster_morgan.csv",
    "KERMT/export_public_cluster_split/all_train_fold_0_cluster_morgan.csv",
    "KERMT/export_public_cluster_split/all_val_fold_0_cluster_morgan.csv",
]

# map from KERMT data to ExpansionRX data on SMILES
map_cols_kermt_to_expansionrx = [
    {
        "input_col": "LogD_pH_7.4",
        "output_col": "LogD (None)",
        "transform": TRANSFORMATIONS["None"],
    },
    {
        "input_col": "kinetic_logSaq",  # log(M)
        "output_col": "KSOL (uM)",
        "transform": TRANSFORMATIONS["10^(x+6)"],
    },
    {
        "input_col": "CL_microsome_human",  # log(mL/min/kg)
        "output_col": "HLM CLint (mL/min/kg)",
        "transform": TRANSFORMATIONS["10^(x)"],
    },
    {
        "input_col": "CL_microsome_mouse",  # log(mL/min/kg)
        "output_col": "MLM CLint (mL/min/kg)",
        "transform": TRANSFORMATIONS["10^(x)"],
    },
    {
        "input_col": "Papp_Caco2",  # log(10^-6 cm/sec)
        "output_col": "Caco-2 Permeability Papp A>B (10^-6 cm/s)",
        "transform": TRANSFORMATIONS["10^(x)"],
    },
    {
        # REVIEW: low confidence mapping
        "input_col": "Pgp_human",  # log(None)
        "output_col": "Caco-2 Permeability Efflux (None)",
        "transform": TRANSFORMATIONS["10^(x)"],
    },
]

Load and concatenate public KERMT files. Drop duplicates and bad data.

In [ ]:
df_kermt_public = pd.DataFrame()
for f in files_biogen:
    df = pd.read_csv(base_data_dir / f)
    df["file"] = f
    df_kermt_public = pd.concat([df_kermt_public, df], ignore_index=True)

# rename smiles --> SMILES
df_kermt_public.rename(columns={"smiles": "SMILES"}, inplace=True)

# drop rows with invalid SMILES
df_kermt_public["SMILES"] = parallel_canonicalize_smiles(
    df_kermt_public["SMILES"].tolist(),
    isomeric=True,
)
n_none_smiles = df_kermt_public["SMILES"].isnull().sum()
if n_none_smiles > 0:
    logger.warning(f"Dropping {n_none_smiles} rows with invalid SMILES")
    df_kermt_public.dropna(subset=["SMILES"], inplace=True)
    df_kermt_public.reset_index(drop=True, inplace=True)

logger.info(f"KERMT Public Dataframe shape: {df_kermt_public.shape}")
logger.info(f"KERMT Public Dataframe columns: {df_kermt_public.columns.tolist()}")

# print finite number of entries for each column
for col in df_kermt_public.columns:
    if not pd.api.types.is_numeric_dtype(df_kermt_public[col]):
        continue
    n_finite = np.isfinite(df_kermt_public[col]).sum()
    logger.info(f"Column '{col}' has {n_finite} finite entries out of {len(df_kermt_public)}")

Transform dataset column names to be compatible with ExpansionRX dataset.

In [ ]:
df_kermt_public_admet_cleaned = df_kermt_public[["SMILES"]]
for mapping in map_cols_kermt_to_expansionrx:
    input_col = mapping["input_col"]
    output_col = mapping["output_col"]
    transform = mapping["transform"]

    df_input = df_kermt_public[["SMILES", input_col]].copy()
    df_input[output_col] = df_input[input_col].apply(transform).copy()
    df_input.drop(columns=[input_col], inplace=True)

    df_kermt_public_admet_cleaned = pd.merge(df_kermt_public_admet_cleaned, df_input, on="SMILES")

Curate dataset to include relevant dynamic ranges for main dataset

In [ ]:
# limit KSol to < 250
idx_infinite_ksol = df_kermt_public_admet_cleaned["KSOL (uM)"].isnull()
idx_too_large = df_kermt_public_admet_cleaned["KSOL (uM)"] > 250.0
idx_remove = ~idx_infinite_ksol & idx_too_large
df_kermt_public_admet_cleaned = df_kermt_public_admet_cleaned[~idx_remove].reset_index(drop=True)

# limit Caco-2 Permeability Papp A>B to < 100
idx_infinite_caco2 = df_kermt_public_admet_cleaned["Caco-2 Permeability Papp A>B (10^-6 cm/s)"].isnull()
idx_too_large_caco2 = df_kermt_public_admet_cleaned["Caco-2 Permeability Papp A>B (10^-6 cm/s)"] > 100.0
idx_remove_caco2 = ~idx_infinite_caco2 & idx_too_large_caco2
df_kermt_public_admet_cleaned = df_kermt_public_admet_cleaned[~idx_remove_caco2].reset_index(drop=True)

# limit Caco-2 Permeability Efflux to < 100
idx_infinite_efflux = df_kermt_public_admet_cleaned["Caco-2 Permeability Efflux (None)"].isnull()
idx_too_large_efflux = df_kermt_public_admet_cleaned["Caco-2 Permeability Efflux (None)"] > 100.0
idx_remove_efflux = ~idx_infinite_efflux & idx_too_large_efflux
df_kermt_public_admet_cleaned = df_kermt_public_admet_cleaned[~idx_remove_efflux].reset_index(drop=True)

# drop rows with all NaN values across ADMET columns
admet_cols = [mapping["output_col"] for mapping in map_cols_kermt_to_expansionrx]
df_kermt_public_admet_cleaned.dropna(subset=admet_cols, how="all", inplace=True)
df_kermt_public_admet_cleaned.reset_index(drop=True, inplace=True)

logger.info(f"Merged KERMT-ExpansionRX Dataframe shape: {df_kermt_public_admet_cleaned.shape}")
logger.info(f"Merged KERMT-ExpansionRX Dataframe columns: {df_kermt_public_admet_cleaned.columns.tolist()}")

In [ ]:
# check for repeated SMILES
n_total = len(df_kermt_public_admet_cleaned)
n_unique = df_kermt_public_admet_cleaned["SMILES"].nunique()
n_duplicates = n_total - n_unique
if n_duplicates > 0:
    logger.warning(f"Found {n_duplicates} duplicate SMILES in merged KERMT-ExpansionRX dataframe")
    # average (skip NaNs) over duplicates
    df_kermt_public_admet_cleaned = df_kermt_public_admet_cleaned.groupby("SMILES", as_index=False).mean()
    logger.info(f"Dataframe shape after averaging duplicates: {df_kermt_public_admet_cleaned.shape}")

# check for overlap with ExpansionRx teaser dataset
smiles_teaser = set(df_expansionrx["SMILES"].tolist())
smiles_kermt_expansionrx = set(df_kermt_public_admet_cleaned["SMILES"].tolist())
overlap_smiles = smiles_teaser.intersection(smiles_kermt_expansionrx)
n_overlap = len(overlap_smiles)
if n_overlap > 0:
    logger.warning(f"Found {n_overlap} overlapping SMILES between ExpansionRx teaser and KERMT-ExpansionRX datasets")

In [ ]:
# print num non-nan entries for each column
for col in df_kermt_public_admet_cleaned.columns:
    if col == "SMILES":
        continue
    n_non_nan = df_kermt_public_admet_cleaned[col].notna().sum()
    logger.info(f"Column '{col}' has {n_non_nan} non-NaN entries out of {len(df_kermt_public_admet_cleaned)}")

In [ ]:
cleaned_data["kermt_public"] = df_kermt_public_admet_cleaned.copy()

Visualize distributions of dataset

In [ ]:
df = df_kermt_public_admet_cleaned.copy()
fname = "kermt_public_admet_cleaned"

output_dir = data_path.parents[1] / "figures"
df_props = compute_molecular_properties(df["SMILES"])
output_dir.mkdir(parents=True, exist_ok=True)
fig, ax = plot_numeric_distributions(
    df,
    n_cols=3,
    title="KERMT Public ADMET Cleaned Property Distributions",
    save_path=output_dir / f"{fname}_numeric_distributions.png",
)
fig, ax = plot_property_distributions(
    df_props,
    save_path=output_dir / f"{fname}_molecular_property_distributions.png",
)
fig, ax = plot_correlation_matrix(
    df,
    save_path=output_dir / f"{fname}_numeric_correlation_matrix.png",
)

#### Biogen

In [ ]:
files_biogen = [
    "KERMT/export_biogen_cluster_split/all_test_cluster_morgan.csv",
    "KERMT/export_biogen_cluster_split/all_train_fold_0_cluster_morgan.csv",
    "KERMT/export_biogen_cluster_split/all_val_fold_0_cluster_morgan.csv",
]

map_cols_biogen_to_expansionrx = [
    {
        "input_col": "SOLY_6.8", # FIXME: This is not pH 7
        "output_col": "KSOL (uM)",
        "transform": TRANSFORMATIONS["ug/mL to uM"],
    },
    {
        "input_col": "MDR1-MDCK_ER", # FIXME: MDCK not Caco-2
        "output_col": "Caco-2 Permeability Efflux (None)",
        "transform": TRANSFORMATIONS["None"],
    },
    {
        "input_col": "HLM_CLint",
        "output_col": "HLM CLint (mL/min/kg)",
        "transform": TRANSFORMATIONS["10^(x)"],
    },
]

In [ ]:
# load and concatenate biogen files
df_kermt_biogen = [pd.read_csv(base_data_dir / file) for file in files_biogen]
df_kermt_biogen = pd.concat(df_kermt_biogen, ignore_index=True)
logger.info(f"KERMT Biogen Dataframe shape: {df_kermt_biogen.shape}")
logger.info(f"KERMT Biogen Dataframe columns: {df_kermt_biogen.columns.tolist()}")

# rename smiles --> SMILES
df_kermt_biogen.rename(columns={"smiles": "SMILES"}, inplace=True)

# drop rows with invalid SMILES
df_kermt_biogen["SMILES"] = parallel_canonicalize_smiles(
    df_kermt_biogen["SMILES"].tolist(),
    isomeric=True,
)
df_kermt_biogen["MW"] = df_kermt_biogen["SMILES"].apply(lambda smi: Descriptors.MolWt(Chem.MolFromSmiles(smi)) if smi is not None else np.nan)

n_none_smiles = df_kermt_biogen["SMILES"].isnull().sum()
if n_none_smiles > 0:
    logger.warning(f"Dropping {n_none_smiles} rows with invalid SMILES")
    df_kermt_biogen.dropna(subset=["SMILES"], inplace=True)
    df_kermt_biogen.reset_index(drop=True, inplace=True)
logger.info(f"KERMT Biogen Dataframe shape after cleaning: {df_kermt_biogen.shape}")


In [ ]:
# map to ExpansionRX columns
df_kermt_biogen_admet_cleaned = df_kermt_biogen[["SMILES"]]
for mapping in map_cols_biogen_to_expansionrx:
    input_col = mapping["input_col"]
    output_col = mapping["output_col"]
    transform = mapping["transform"]

    df_input = df_kermt_biogen[["SMILES", input_col]].copy()
    if input_col == "SOLY_6.8":
        # need MW for conversion
        df_input = pd.merge(
            df_input,
            df_kermt_biogen[["SMILES", "MW"]],
            on="SMILES",
            how="left",
        )
        df_input[output_col] = df_input.apply(
            lambda row: transform(row[input_col], row["MW"]),
            axis=1,
        )
        df_input.drop(columns=["MW"], inplace=True)
    else:
        df_input[output_col] = df_input[input_col].apply(transform).copy()
        
    logger.info(f"Transformed column '{input_col}' to '{output_col}' with {len(df_input)} entries.")
    df_input.drop(columns=[input_col], inplace=True)

    df_kermt_biogen_admet_cleaned = pd.merge(df_kermt_biogen_admet_cleaned, df_input, on="SMILES")

In [ ]:
# print num non-nan entries for each column
for col in df_kermt_biogen_admet_cleaned.columns:
    if col == "SMILES":
        continue
    n_non_nan = df_kermt_biogen_admet_cleaned[col].notna().sum()
    logger.info(f"Column '{col}' has {n_non_nan} non-NaN entries out of {len(df_kermt_biogen_admet_cleaned)}")

In [ ]:
# export data
logger.info(f"KERMT Biogen-ExpansionRX Dataframe shape: {df_kermt_biogen_admet_cleaned.shape}")
cleaned_data["kermt_biogen"] = df_kermt_biogen_admet_cleaned.copy()

In [ ]:
df = df_kermt_biogen_admet_cleaned.copy()
output_dir = data_path.parents[1] / "figures"
fname = "kermt_biogen_admet_cleaned"

df_props = compute_molecular_properties(df["SMILES"])
output_dir.mkdir(parents=True, exist_ok=True)

fig, ax = plot_numeric_distributions(
    df,
    n_cols=3,
    title="KERMT Biogen ADMET Cleaned Property Distributions",
    save_path=output_dir / f"{fname}_numeric_distributions.png",
)
fig, ax = plot_property_distributions(
    df_props,
    save_path=output_dir / f"{fname}_molecular_property_distributions.png",
)
fig, ax = plot_correlation_matrix(
    df,
    save_path=output_dir / f"{fname}_numeric_correlation_matrix.png",
)


### PharmaBench

In [ ]:
files_pharmabench = [
    "KERMT/export_biogen_cluster_split/all_test_cluster_morgan.csv",
    "KERMT/export_biogen_cluster_split/all_train_fold_0_cluster_morgan.csv",
    "KERMT/export_biogen_cluster_split/all_val_fold_0_cluster_morgan.csv",
]

map_cols_pharmabench_to_expansionrx = [
    {
        "input_col": "SOLY_6.8",  # FIXME: This is not pH 7
        "output_col": "KSOL (uM)",
        "transform": TRANSFORMATIONS["ug/mL to uM"],
    },
    {
        "input_col": "MDR1-MDCK_ER",  # FIXME: MDCK not Caco-2
        "output_col": "Caco-2 Permeability Efflux (None)",
        "transform": TRANSFORMATIONS["None"],
    },
    {
        "input_col": "HLM_CLint", # log10(mL/min/g)
        "output_col": "HLM CLint (mL/min/kg)",
        "transform": TRANSFORMATIONS["10^(x)"],
    },
]

### Admetica

In [ ]:
# TODO: Implement loading and processing

### TDC

In [ ]:
# TODO: Implement loading and processing

### NCATS

In [ ]:
# TODO: Implement loading and processing

### ChEMBL

In [ ]:
# TODO: Implement loading and processing

## Final

In [ ]:
# Save all cleaned datasets as a single dataframe with column for dataset name
df_out = pd.DataFrame()
for name, df in cleaned_data.items():
    logger.info(f"Adding cleaned dataset '{name}' with shape {df.shape} to combined dataframe")
    df_temp = df.copy()
    df_temp["dataset"] = name
    df_out = pd.concat([df_out, df_temp], axis=0)
logger.info(f"Combined cleaned dataframe shape: {df_out.shape}")
df_out.reset_index(drop=True, inplace=True)
    
# limit datasets to physically reasonable ranges
df_props = compute_molecular_properties(df_out["SMILES"])
idx_drop = df_props["MW"] > 1000.0
idx_drop |= df_props["LogP"] < -5.0
idx_drop |= df_props["LogP"] > 10.0
idx_drop |= df_props["TPSA"] > 300.0
idx_drop |= df_props["HBA"] > 20
idx_drop |= df_props["HBD"] > 10
idx_drop |= df_props["RotBonds"] > 20
idx_drop |= df_props["NumRings"] > 10
# remove any idx_drop that are in the ExpansionRx teaser dataset
idx_teaser = df_out["dataset"] == "expansionrx"
idx_drop = idx_drop & ~idx_teaser
# drop entries
n_dropped = idx_drop.sum()
if n_dropped > 0:
    logger.warning(f"Dropping {n_dropped} entries from combined dataset due to unreasonable molecular properties")
    df_out = df_out.loc[~idx_drop].copy()
    
for name in cleaned_data.keys():
    df_subset = df_out[df_out["dataset"] == name]

    # check for repeated SMILES
    n_total = len(df_subset)
    n_unique = df_subset["SMILES"].nunique()
    n_duplicates = n_total - n_unique
    if n_duplicates > 0:
        logger.warning(f"Found {n_duplicates} duplicate SMILES in combined dataset for dataset '{name}'")
        # average (skip NaNs) over duplicates
        df_subset = df_subset.groupby("SMILES", as_index=False).mean()
        # update in main dataframe
        df_out = pd.concat(
            [df_out[df_out["dataset"] != name], df_subset],
            axis=0,
        )
        logger.info(f"Dataframe shape after averaging duplicates for dataset '{name}': {df_out.shape}")
        
        # look for rows where all ADMET values are NaN
        admet_cols = [col for col in df_subset.columns if col not in ["SMILES", "dataset"]]
        n_before = len(df_out)
        df_out = df_out.dropna(subset=admet_cols, how="all")
        n_after = len(df_out)
        n_dropped = n_before - n_after
        if n_dropped > 0:
            logger.warning(f"Dropped {n_dropped} rows with all NaN ADMET values for dataset '{name}' after averaging duplicates")
            df_out = df_out[df_out["dataset"] != name]
            df_out = pd.concat([df_out, df_subset], axis=0)

# look at overlap between datasets
dataset_names = list(cleaned_data.keys())
for i in range(len(dataset_names)):
    name_i = dataset_names[i]
    smiles_i = set(df_out[df_out["dataset"] == name_i]["SMILES"].tolist())
    for j in range(i + 1, len(dataset_names)):
        name_j = dataset_names[j]
        smiles_j = set(df_out[df_out["dataset"] == name_j]["SMILES"].tolist())
        overlap_smiles = smiles_i.intersection(smiles_j)
        n_overlap = len(overlap_smiles)
        if n_overlap > 0:
            logger.warning(f"Found {n_overlap} overlapping SMILES between datasets '{name_i}' and '{name_j}'")
            
# for any repeated SMILES across datasets, keep only the first occurrence, prioritizing
# ExpansionRx teaser > KERMT Public > KERMT Biogen > Pharmabench
df_out.sort_values(by="dataset", inplace=True)
df_out.drop_duplicates(subset=["SMILES"], keep="first", inplace=True) 

# save final combined cleaned dataset
df_out.reset_index(drop=True, inplace=True)
output_path = base_data_dir.parent / "cleaned_combined_datasets.csv"
df_out.to_csv(output_path, index=False)

In [ ]:
# plot the distribution of dataset counts
dataset_counts = df_out["dataset"].value_counts()
fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(
    x=dataset_counts.index,
    y=dataset_counts.values,
    ax=ax,
    palette="viridis",
    hue=dataset_counts.index,
    dodge=False,
)
ax.set_xlabel("Dataset", fontsize=14)
ax.set_ylabel("Count", fontsize=14)
ax.set_title("Distribution of Dataset Counts", fontsize=18)
ax.tick_params(axis="x", labelrotation=30, labelsize=12)
ax.set_yscale("log")
fig.tight_layout()

In [ ]:
# plot the distribution of targets colored by dataset
target_cols = [col for col in df_out.columns if col != "SMILES" and col != "dataset"]

colormap = cc.glasbey  # categorical colormap
dataset_to_color = {name: colormap[i] for i, name in enumerate(df_out["dataset"].unique())}

for target_col in target_cols:
    if not pd.api.types.is_numeric_dtype(df_out[target_col]):
        continue
    fig, ax = plt.subplots(figsize=(8, 6))
    logger.info(f"Plotting distribution for target column '{target_col}'")
    for dataset_name, group in df_out.groupby("dataset"):
        sns.kdeplot(
            group[target_col].dropna(),
            ax=ax,
            label=dataset_name,
            fill=True,
            alpha=0.5,
            color=dataset_to_color[dataset_name],
        )
    ax.set_xlabel(target_col, fontsize=14)
    ax.set_ylabel("Density", fontsize=14)
    ax.set_title(f"Distribution of {target_col} by Dataset", fontsize=18)
    ax.tick_params(axis="x", labelrotation=30, labelsize=12)
    ax.legend(title="Dataset")
    fig.tight_layout()
    
    fig.savefig(
        output_dir / f"combined_{target_col.replace(' ', '_').replace('(', '').replace(')', '').replace('/', '_to_')}_distribution.png",
        dpi=300,
    )



In [ ]:
# plot distribution of molecular properties colored by dataset
df_props = compute_molecular_properties(df_out["SMILES"])
df_props["dataset"] = df_out["dataset"]
for prop_col in df_props.columns:
    if prop_col == "SMILES" or prop_col == "dataset":
        continue
    fig, ax = plt.subplots(figsize=(8, 6))
    logger.info(f"Plotting distribution for molecular property '{prop_col}'")
    for dataset_name, group in df_props.groupby("dataset"):
        sns.kdeplot(
            group[prop_col].dropna(),
            ax=ax,
            label=dataset_name,
            fill=True,
            alpha=0.5,
            color=dataset_to_color[dataset_name],
        )
    ax.set_xlabel(prop_col, fontsize=14)
    ax.set_ylabel("Density", fontsize=14)
    ax.set_title(f"Distribution of {prop_col} by Dataset", fontsize=18)
    ax.tick_params(axis="x", labelrotation=30, labelsize=12)
    ax.legend(title="Dataset")
    fig.tight_layout()
    
    fig.savefig(
        output_dir / f"combined_{prop_col.replace(' ', '_').replace('(', '').replace(')', '')}_distribution.png",
        bbox_inches="tight",
        dpi=300,
    )

In [ ]:
from skfp.fingerprints import ECFPFingerprint
from skfp.preprocessing import MolFromSmilesTransformer

mol_from_smiles = MolFromSmilesTransformer(suppress_warnings=True)
mols = mol_from_smiles.transform(df_out["SMILES"].tolist())

fpgen = ECFPFingerprint(count=False, radius=2, fp_fize=2048)
fps = fpgen.transform(mols)

In [ ]:
from skfp.fingerprints import ECFPFingerprint
from skfp.distances import tanimoto_binary_similarity

In [ ]:
# plot the distribution of Tanimoto similarities to the reference set (ExpansionRx teaser)
fpgen = ECFPFingerprint(count=False)
fps = fpgen.transform(df_out["SMILES"].tolist())

# compute Tanimoto similarities to reference set
fps_ref = fps[df_out["dataset"] == "expansionrx"]
for dataset_name, group in df_out.groupby("dataset"):
    if dataset_name == "expansionrx":
        continue
   
    fps_group = fps[group.index.tolist()]
    tanimoto_sims = np.zeros((len(fps_group), len(fps_ref)))
    for i in range(len(fps_group)):
        for j in range(len(fps_ref)):
            tanimoto_sims[i, j] = fpgen.tanimoto(fps_group[i], fps_ref[j])
    max_sims = tanimoto_sims.max(axis=1)
    
    # plot distribution of max Tanimoto similarities
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.kdeplot(
        max_sims,
        ax=ax,
        fill=True,
        alpha=0.5,
        color=dataset_to_color[dataset_name],
    )
    ax.set_xlabel("Max Tanimoto Similarity to ExpansionRx Teaser", fontsize=14)
    ax.set_ylabel("Density", fontsize=14)
    ax.set_title(f"Tanimoto Similarity Distribution for {dataset_name}", fontsize=18)
    ax.tick_params(axis="x", labelrotation=30, labelsize=12)
    fig.tight_layout()
    
    fig.savefig(
        output_dir / f"combined_{dataset_name}_tanimoto_similarity_distribution.png",
        dpi=300,
    )

In [ ]:
# TODO: look at SMILES overlap between datasets